In [3]:
%load_ext kedro.ipython

[10/15/24 09:18:21] INFO     Registered line magic '%reload_kedro'                                   ]8;id=188617;file:///home/seanlandry/dev/kgql-client/venv/lib64/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=578623;file:///home/seanlandry/dev/kgql-client/venv/lib64/python3.11/site-packages/kedro/ipython/__init__.py#61\61]8;;\

                    INFO     Registered line magic '%load_node'                                      ]8;id=479189;file:///home/seanlandry/dev/kgql-client/venv/lib64/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=170751;file:///home/seanlandry/dev/kgql-client/venv/lib64/python3.11/site-packages/kedro/ipython/__init__.py#63\63]8;;\

                    INFO     Resolved project path as:                                              ]8;id=319679;file:///home/seanlandry/dev/kgql-client/venv/lib64/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=990246;file:///home/seanlandry/dev/kgql-client/venv/lib64/python3.11/site-packages/kedro/ipython/__init__.py#178\178]8;;\
                             /home/seanlandry/dev/kgql-client/kedro-graphql.                                       
                             To set a different path, run '%reload_kedro <project_root>'                           

[10/15/24 09:18:22] INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=523500;file:///home/seanlandry/dev/kgql-client/venv/lib64/python3.11/site-packages/kedro_telemetry/plugin.py\plugin.py]8;;\:]8;id=266764;file:///home/seanlandry/dev/kgql-client/venv/lib64/python3.11/site-packages/kedro_telemetry/plugin.py#233\233]8;;\
                             the product. No personal data or IP addresses are stored on our side. If              
                             you want to opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK`              
                             environment variables, or create a `.telemetry` file in the current                   
                             working directory with the contents `consent: false`. Read more at                    
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

                    INFO     Kedro project kedro-graphql                                            ]8;id=533613;file:///home/seanlandry/dev/kgql-client/venv/lib64/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=797193;file:///home/seanlandry/dev/kgql-client/venv/lib64/python3.11/site-packages/kedro/ipython/__init__.py#144\144]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=573397;file:///home/seanlandry/dev/kgql-client/venv/lib64/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=122972;file:///home/seanlandry/dev/kgql-client/venv/lib64/python3.11/site-packages/kedro/ipython/__init__.py#145\145]8;;\
                             'pipelines'                                                                           

# Define a KedroGraphqlClient

In [122]:
# src/kedro_graphql/client.py

import asyncio

from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
from gql.transport.websockets import WebsocketsTransport
from kedro_graphql.models import PipelineInput, Pipeline, PipelineEvent, PipelineLogMessage
from fastapi.encoders import jsonable_encoder
from strawberry.utils.str_converters import to_camel_case, to_snake_case

class KedroGraphqlClient():

    def __init__(self, url = None, ws = None ):
        """
        Kwargs:
            url (str): Url to api [default: http://localhost:5000/graphql]
        
        """
        self.url = url or "http://localhost:5000/graphql"
        self.ws = ws or "ws://localhost:5000/graphql"
        self.aio_transport = AIOHTTPTransport(url=self.url) 
        self.web_transport = WebsocketsTransport(url=self.ws) 

    async def createPipeline(self, pipeline: PipelineInput):
    
        # Using `async with` on the client will start a connection on the transport
        # and provide a `session` variable to execute queries on this connection
        async with Client(
            transport=self.aio_transport,
        ) as gql_session:
    
            # Execute mutation
            query = gql(
                """
                mutation createPipeline($pipeline: PipelineInput!) {
                  pipeline(pipeline: $pipeline) {
                    id
                    name
                    describe
                    dataCatalog {
                      name
                      config
                    }
                    inputs {
                      name
                      config
                    }
                    nodes {
                      name
                      inputs
                      outputs
                      tags
                    }
                    outputs {
                      name
                      config
                    }
                    parameters {
                      name
                      value
                    }
                    status
                    tags {
                      key
                      value
                    }
                    taskId
                    taskName
                    taskArgs
                    taskKwargs
                    taskRequest
                    taskException
                    taskTraceback
                    taskEinfo
                    taskResult
                  }
                }
            """
            )

            ## convert from object to json compatible dict
            pipeline = jsonable_encoder(pipeline)

            ## need to convert to camelCase
            pipeline = {to_camel_case(k):v for k,v in pipeline.items()}
                        
            result = await gql_session.execute(query, variable_values={"pipeline":pipeline})
            return Pipeline.from_dict(result["pipeline"])

    async def readPipeline(self, pipeline: str):
    
        # Using `async with` on the client will start a connection on the transport
        # and provide a `session` variable to execute queries on this connection
        async with Client(
            transport=self.aio_transport,
        ) as gql_session:
    
            # Execute query
            query = gql(
                """
                query readPipeline($pipeline: String!) {
                  pipeline(id: $pipeline) {
                    id
                    name
                    describe
                    dataCatalog {
                      name
                      config
                    }
                    inputs {
                      name
                      config
                    }
                    nodes {
                      name
                      inputs
                      outputs
                      tags
                    }
                    outputs {
                      name
                      config
                    }
                    parameters {
                      name
                      value
                    }
                    status
                    tags {
                      key
                      value
                    }
                    taskId
                    taskName
                    taskArgs
                    taskKwargs
                    taskRequest
                    taskException
                    taskTraceback
                    taskEinfo
                    taskResult
                  }
                }
            """
            )
                        
            result = await gql_session.execute(query, variable_values={"pipeline":pipeline})
            result = {to_snake_case(k):v for k,v in result["pipeline"].items()}
            return Pipeline.from_dict(result)

    async def pipelineEvents(self, pipeline: str):
    
        # Using `async with` on the client will start a connection on the transport
        # and provide a `session` variable to execute queries on this connection
        async with Client(
            transport=self.web_transport,
        ) as gql_session:
    
            # Execute
            query = gql(
                """
                subscription pipelineEvents($pipeline: String!) {
                  pipeline(id: $pipeline) {
                    id
                    taskId
                    status
                    result
                    timestamp
                    traceback
                  }
                }
            """
            )

            async for result in gql_session.subscribe(query, variable_values={"pipeline":pipeline}):
                result = {to_snake_case(k):v for k,v in result["pipeline"].items()}
                yield PipelineEvent(**result)

    async def pipelineLogs(self, pipeline: str):
    
        # Using `async with` on the client will start a connection on the transport
        # and provide a `session` variable to execute queries on this connection
        async with Client(
            transport=self.web_transport,
        ) as gql_session:
    
            # Execute
            query = gql(
                """
                subscription pipelineLogs($pipeline: String!) {
                  pipeline(id: $pipeline) {
                    id
                    message
                    messageId
                    taskId
                    time
                  }
                }
            """
            )

            async for result in gql_session.subscribe(query, variable_values={"pipeline":pipeline}):
                result = {to_snake_case(k):v for k,v in result["pipeline"].items()}
                ## need to figure out which fields can be optional
                print(result)
                yield PipelineLogMessage(id = result["id"], 
                                         message = result.get("message", ""),
                                         message_id = result.get("message_id", ""),
                                         task_id = result.get("task_id", ""),
                                         time = result.get("time", ""))


# Usage

In [126]:
client = KedroGraphqlClient()

## Mutations

### Create a pipeline

In [127]:
import json

input_dict = {"type": "text.TextDataset", "filepath": "/home/seanlandry/dev/kgql-client/kedro-graphql/data/01_raw/text_in.txt"}
output_dict = {"type": "text.TextDataset", "filepath": "/home/seanlandry/dev/kgql-client/kedro-graphql/data/02_intermediate/text_out.txt"}


## PipelineInput object
p = PipelineInput(**{
                  "name": "example00",
                  "data_catalog":[{"name": "text_in", "config": json.dumps(input_dict)},
                                  {"name": "text_out", "config": json.dumps(output_dict)}],
                  "parameters": [{"name":"example", "value":"hello"},
                                 {"name": "duration", "value": "0.1", "type": "FLOAT"}],
                  "tags": [{"key": "author", "value": "opensean"},{"key":"package", "value":"kedro-graphql"}]
                 })



result00 = await client.createPipeline(p)

In [128]:
result00


Pipeline(
    kedro_pipelines_index=None,
    id='670f3fbdd9d965459c566b83',
    inputs=None,
    name='example00',
    outputs=None,
    data_catalog=None,
    parameters=[
        Parameter(name='example', value='hello', type=<ParameterType.STRING: 'string'>),
        Parameter(name='duration', value='0.1', type=<ParameterType.STRING: 'string'>)
    ],
    status='PENDING',
    tags=[Tag(key='author', value='opensean'), Tag(key='package', value='kedro-graphql')],
    task_id=None,
    task_name=None,
    task_args=None,
    task_kwargs=None,
    task_request=None,
    task_exception=None,
    task_traceback=None,
    task_einfo=None,
    task_result=None
)

## Queries

### Read a pipeline

In [129]:
result01 = await client.readPipeline(result00.id)
result01


Pipeline(
    kedro_pipelines_index=None,
    id='670f3fbdd9d965459c566b83',
    inputs=None,
    name='example00',
    outputs=None,
    data_catalog=[
        DataSet(
            name='text_in',
            config='{"type": "text.TextDataset", "filepath": "/home/seanlandry/dev/kgql-client/kedro-graphql/data/01_raw/text_in.txt"}',
            type=None,
            filepath=None,
            save_args=None,
            load_args=None,
            credentials=None
        ),
        DataSet(
            name='text_out',
            config='{"type": "text.TextDataset", "filepath": "/home/seanlandry/dev/kgql-client/kedro-graphql/data/02_intermediate/text_out.txt"}',
            type=None,
            filepath=None,
            save_args=None,
            load_args=None,
            credentials=None
        )
    ],
    parameters=[
        Parameter(name='example', value='hello', type=<ParameterType.STRING: 'string'>),
        Parameter(name='duration', value='0.1', type=<ParameterType

## Subscriptions

### Subscribe to Pipeline Events

In [130]:
async for e in client.pipelineEvents(result00.id):
    print(e)

PipelineEvent(id='670f3fbdd9d965459c566b83', task_id='fb41f92c-a4a5-4aae-bd0b-27f4beeeca6a', status='SUCCESS', result='success', timestamp='1729052612.7779524', traceback=None)


### Subscribe to Pipeline Logs

In [131]:
async for e in client.pipelineLogs(result00.id):
    print(e)

{'id': '670f3fbdd9d965459c566b83', 'task_id': 'fb41f92c-a4a5-4aae-bd0b-27f4beeeca6a'}
PipelineLogMessage(id='670f3fbdd9d965459c566b83', message='', message_id='', task_id='fb41f92c-a4a5-4aae-bd0b-27f4beeeca6a', time='')


## Full Example

In [132]:
## create a pipeline
import json

input_dict = {"type": "text.TextDataset", "filepath": "/home/seanlandry/dev/kgql-client/kedro-graphql/data/01_raw/text_in.txt"}
output_dict = {"type": "text.TextDataset", "filepath": "/home/seanlandry/dev/kgql-client/kedro-graphql/data/02_intermediate/text_out.txt"}


## PipelineInput object
p = PipelineInput(**{
                  "name": "example00",
                  "data_catalog":[{"name": "text_in", "config": json.dumps(input_dict)},
                                  {"name": "text_out", "config": json.dumps(output_dict)}],
                  "parameters": [{"name":"example", "value":"hello"},
                                 {"name": "duration", "value": "1", "type": "FLOAT"}],
                  "tags": [{"key": "author", "value": "opensean"},{"key":"package", "value":"kedro-graphql"}]
                 })



result00 = await client.createPipeline(p)
print("created pipeline with id:", result00.id) 

## subscribe to logs
print("streaming logs")
async for e in client.pipelineLogs(result00.id):
    print(e)
    
## read result
print("pipeline complete")
result01 = await client.readPipeline(result00.id)
result01

created pipeline with id: 670f3fc8d9d965459c566b84
streaming logs
{'id': '670f3fc8d9d965459c566b84', 'task_id': 'ce303532-aaa7-4ad5-9259-71c47e85d946'}
PipelineLogMessage(id='670f3fc8d9d965459c566b84', message='', message_id='', task_id='ce303532-aaa7-4ad5-9259-71c47e85d946', time='')
{'id': '670f3fc8d9d965459c566b84', 'task_id': 'ce303532-aaa7-4ad5-9259-71c47e85d946'}
PipelineLogMessage(id='670f3fc8d9d965459c566b84', message='', message_id='', task_id='ce303532-aaa7-4ad5-9259-71c47e85d946', time='')
{'id': '670f3fc8d9d965459c566b84', 'task_id': 'ce303532-aaa7-4ad5-9259-71c47e85d946'}
PipelineLogMessage(id='670f3fc8d9d965459c566b84', message='', message_id='', task_id='ce303532-aaa7-4ad5-9259-71c47e85d946', time='')
pipeline complete



Pipeline(
    kedro_pipelines_index=None,
    id='670f3fc8d9d965459c566b84',
    inputs=None,
    name='example00',
    outputs=None,
    data_catalog=[
        DataSet(
            name='text_in',
            config='{"type": "text.TextDataset", "filepath": "/home/seanlandry/dev/kgql-client/kedro-graphql/data/01_raw/text_in.txt"}',
            type=None,
            filepath=None,
            save_args=None,
            load_args=None,
            credentials=None
        ),
        DataSet(
            name='text_out',
            config='{"type": "text.TextDataset", "filepath": "/home/seanlandry/dev/kgql-client/kedro-graphql/data/02_intermediate/text_out.txt"}',
            type=None,
            filepath=None,
            save_args=None,
            load_args=None,
            credentials=None
        )
    ],
    parameters=[
        Parameter(name='example', value='hello', type=<ParameterType.STRING: 'string'>),
        Parameter(name='duration', value='1', type=<ParameterType.S